---
<p align="center">
  <img src="https://github.com/lacamposm/course-fundamentals-llms-openai-langchain/raw/main/images/image_igac.jpg" alt="Imagen_IGAC" width="280">
</p>

---

# ***Fundamentos de LLMs con Python: Explorando ChatGPT y LangChain***

---

#### ***Instructor: [Luis Andrés Campos Maldonado](https://www.linkedin.com/in/lacamposm/)***

##### ***Email: luisandres.campos@igac.gov.co***

##### ***Contratista-Observatorio Inmobiliario Catastral***

---

# ***Clase 10 - 10 de mayo de 2024***
---

## ***Taller Práctico de Implementación con Streamlit - Parte 2***

**Objetivos de Aprendizaje:**

- Continuar   y   finalizar   el   desarrollo   de   la   aplicación   Streamlit, integrando todas las funcionalidades discutidas durante el curso.

- Realizar   pruebas,   depuración   y   optimización   de   la   aplicación desarrollada.

In [1]:
%%capture
!pip install pandas openpyxl langchain openai langchain-openai langchain-community langchain-core langchain-text-splitters 
!pip install streamlit chromadb pypdf
!pip install -U duckduckgo_search==5.3.0b4

In [1]:
%%writefile app_final_proyect_agent.py
import os
import shutil
import streamlit as st
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.output_parsers import StrOutputParser
## Tools and agents
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.tools.retriever import create_retriever_tool
from langchain.memory import ConversationBufferMemory
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)


os.environ["OPENAI_API_KEY"] = "<YOUR_API_KEY>"
save_rag_directory = "RAG"


def save_uploaded_files(uploaded_files):
    """
    Function that saves uploaded files in a specific directory and creates a document retriever.
    
    :param uploaded_files: list of files uploaded by the user.
    :param save_rag_directory: directory where the files will be saved.
    
    Returns: None    
    """
    if len(uploaded_files) > 0:
        os.makedirs(save_rag_directory, exist_ok=True)
        for uploaded_file in uploaded_files:
            bytes_data = uploaded_file.read()
            path_file = os.path.join(save_rag_directory, uploaded_file.name)
            with open(path_file, "wb") as f:
                f.write(bytes_data)
                st.write(f"Archivo {uploaded_file.name} guardado")

        st.session_state["retriever"] = create_retriever_documents()
        st.session_state["agent_executor"] = create_agent_chat_with_memory()
        st.rerun()
    else:
        st.rerun()

def chatbot_template(system_message, user_question):
    """
    Create a template to pass custom queries to LLM 
    :param user_question: Message user.
    :param system_message: Message to the system, like a rol.
    :return: Configured with the system message, chat history, and user question.
    """
    chat_prompt = ChatPromptTemplate(
        messages=[
            SystemMessagePromptTemplate.from_template(system_message),
            MessagesPlaceholder(variable_name="chat_history"),
            HumanMessagePromptTemplate.from_template(user_question)
        ]
    )
    return chat_prompt


def create_retriever_documents():
    """
    Creates a document retriever from PDF files stored in a specified directory.

    This function loads documents from a directory using PyPDFDirectoryLoader, splits the
    text into manageable segments using RecursiveCharacterTextSplitter, and then embeds
    these segments using OpenAI's text embedding model. The embedded text segments are
    stored in a vector storage (Chroma), which is then used to create a retriever for
    efficient document retrieval based on vector similarity.

    Return:
        Retriever: An object capable of retrieving documents based on query similarity.
    """
    loader = PyPDFDirectoryLoader(save_rag_directory)
    docs = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    openai_embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
    documents = text_splitter.split_documents(docs)
    vectorstore = Chroma.from_documents(documents, embedding=openai_embeddings)
    retriever = vectorstore.as_retriever()

    return retriever


def stream_response_with_memory_openai(model_name, query, chat_history):
    """
    Streams a response for a given query using an OpenAI model, formatted to maintain conversation history.

    :param model_name: The name of the OpenAI model to use for generating responses.
    :param query: The current user question or input.
    :param chat_history: The history of previous interactions to be considered for context.
    :return: A streaming object that continuously provides the chatbot's responses.
    """
    system_prompt = """You are a friendly chatbot having a conversation with a human and giving answers only in \
    Spanish."""
    user_question = "{user_question}"
    prompt = chatbot_template(system_message=system_prompt, user_question=user_question)
    chat_openai = ChatOpenAI(model=model_name)

    chain = (
            {"chat_history": lambda x: chat_history, "user_question": RunnablePassthrough()}
            | prompt
            | chat_openai
            | StrOutputParser()
    )

    return chain.stream(query)

def create_agent_chat_with_memory():
    """
    Creates and configures a chat agent with memory capabilities integrating specific tools for search and
    information retrieval, using a language model for generating responses.

    This function sets up a DuckDuckGo-based search tool and a retrieval tool configured for inquiries related
    to multipurpose cadastre in Colombia. It also initializes a conversation memory that stores and retrieves
    past conversation messages to maintain context across user interactions.

    :param None: This function does not take any parameters.

    :return:
        AgentExecutor: An executor that manages the lifecycle of the agent, including tool execution
        and conversation memory management.

    Example:
        agent_executor = create_agent_chat_with_memory()
        response = agent_executor.invoke({"user_message": "What is a multipurpose cadastre?"})
        print(response["output"])  # Displays the agent's response to the user's question.
    """
    # Search tool.
    search_wrapper = DuckDuckGoSearchAPIWrapper()
    search_tool = DuckDuckGoSearchRun(api_wrapper=search_wrapper)

    # Retrival tool.
    retriever_tool = create_retriever_tool(
        retriever=st.session_state["retriever"],
        name="catastro_multiproposito",
        description="""Search for information on multipurpose cadastre in Colombia. 
        If you have any questions about multipurpose cadastre in Colombia, you should use this tool!"""
    )
    
    tools = [search_tool, retriever_tool]
    memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")
    llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

    prompt = ChatPromptTemplate(
        messages=[
            SystemMessagePromptTemplate.from_template(
                "Eres un buen chatbot teniendo una conversación con un humano."
            ),
            MessagesPlaceholder(variable_name="chat_history"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
            HumanMessagePromptTemplate.from_template("{user_message}")
        ]
    )

    agent = create_tool_calling_agent(llm,tools,prompt)
    agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=True)
    
    return agent_executor

# =================================== Interfaz de usuario ============================================ #
st.title("🦜 Agent - WebSearch and RAG 🦜")

if "chat_active" not in st.session_state:
    with st.sidebar.expander(f"Seleccione modelo y cargue sus archivos."):
        model_openai = st.radio(
            "Seleccione su ChatModel:",
            ["gpt-3.5-turbo-0125", "gpt-4-0125-preview"],
            key="openai_chat_model"
        )
        st.session_state["openai_model"] = model_openai
        if os.path.exists(save_rag_directory):
            delete_dir = st.button(f"Borrar carpeta {save_rag_directory}")
            if delete_dir:
                shutil.rmtree(save_rag_directory)
                st.rerun()

        uploaded_files = st.file_uploader("Elige los archivos PDF para el RAG", accept_multiple_files=True,
                                          type=["pdf"])

        if st.button("Finalizar carga de archivos y Chatear"):
            st.session_state["chat_active"] = True
            save_uploaded_files(uploaded_files)


if "chat_active" in st.session_state and "agent_executor" in st.session_state:
    st.text(f"Archivos cargados para chatear:\n {', '.join(os.listdir(save_rag_directory))}")

if "chat_history" not in st.session_state:
    st.session_state.chat_history = []

for message in st.session_state.chat_history:
    if isinstance(message, HumanMessage):
        with st.chat_message("Human"):
            st.markdown(message.content)
    else:
        with st.chat_message("AI"):
            st.markdown(message.content)

user_query = st.chat_input("Your message")

if user_query is not None and user_query != "":
    st.session_state.chat_history.append(HumanMessage(user_query))

    with st.chat_message("Human"):
        st.markdown(user_query)

    with st.chat_message("AI"):
            if os.path.exists(save_rag_directory) and "retriever" in st.session_state:
                agent_executor = st.session_state["agent_executor"]
                agent_response = agent_executor.invoke({"user_message": user_query})["output"]
                st.write(agent_response)
            
            else:
                st.warning("No se está usando la capacidad del agente!")
                agent_response = st.write_stream(
                    stream_response_with_memory_openai(
                        model_name=st.session_state["openai_model"],
                        query=user_query,
                        chat_history=st.session_state.chat_history)
                )
    st.session_state.chat_history.append(AIMessage(agent_response))

Writing app_final_proyect_agent.py


In [ ]:
!npm install localtunnel

In [ ]:
!streamlit run /content/app_final_proyect_agent.py &>/content/logs.txt &

In [ ]:
!npx localtunnel --port 8501